# Core10_13 — Handoff Bundle Export

## 목적
Core10의 핵심 산출물을
- 연구 노트북이 아닌
- **프로그램 이식 가능한 번들(handoff bundle)**

형태로 정리한다.

## Bundle 구성
- shutdown_contract.json
- fallback_params.json
- reservation_params.json
- allocation_policy.json
- operation_rollout_log.csv
- metrics_summary.csv

## 산출물
/handoff/core10_bundle/

In [11]:
from pathlib import Path
import shutil
import json
import pandas as pd
from datetime import datetime

# Root paths
ART_ROOT = Path("../artifact")
ART_CORE10 = ART_ROOT / "core10"

HANDOFF_ROOT = ART_ROOT / "handoff"
BUNDLE_DIR = HANDOFF_ROOT / "core10_bundle"

BUNDLE_DIR.mkdir(parents=True, exist_ok=True)

print("handoff bundle dir:")
print(BUNDLE_DIR.resolve())

handoff bundle dir:
/Users/mac/Desktop/De/Developability_Data/core/artifact/handoff/core10_bundle


In [12]:
BUNDLE_FILES = {
    "shutdown_contract": ART_CORE10 / "shutdown_contract.json",
    "fallback_params": ART_CORE10 / "fallback_params.json",
    "reservation_params": ART_CORE10 / "reservation_params.json",
    "allocation_policy": ART_CORE10 / "core10_06_allocation_policy.json",
    "operation_rollout_log": ART_CORE10 / "core10_07_operation_rollout_log.csv",
    "metrics_summary": ART_CORE10 / "core10_10_metrics_summary.csv",
}

In [13]:
from pathlib import Path
import json
from datetime import datetime

ART_CORE10 = Path("../artifact/core10")
ART_CORE10.mkdir(parents=True, exist_ok=True)

shutdown_contract = {
    "contract_id": "core10_shutdown_v1",
    "purpose": "Hard termination guard for governed allocation system",
    "trigger_conditions": {
        "manual_shutdown": True,
        "redesign_request_exceeded": True,
        "external_override": True
    },
    "guarantees": {
        "no_optimization_after_shutdown": True,
        "allocation_frozen": True,
        "audit_log_persisted": True
    },
    "created_at": datetime.utcnow().isoformat()
}

path = ART_CORE10 / "shutdown_contract.json"
path.write_text(json.dumps(shutdown_contract, indent=2), encoding="utf-8")

path

PosixPath('../artifact/core10/shutdown_contract.json')

In [14]:
fallback_params = {
    "policy": "signature_representative_fallback",
    "selection_rule": {
        "sort": [
            "proxy_survivability_score desc",
            "tie_break_risk asc",
            "antibody_id asc"
        ],
        "per_signature_limit": 1
    },
    "safety_guarantee": {
        "no_model_training": True,
        "deterministic": True
    },
    "created_at": datetime.utcnow().isoformat()
}

path = ART_CORE10 / "fallback_params.json"
path.write_text(json.dumps(fallback_params, indent=2), encoding="utf-8")

path

PosixPath('../artifact/core10/fallback_params.json')

In [15]:
reservation_params = {
    "reservation_enabled": True,
    "ttl_steps": 8,
    "cooldown_steps": 5,
    "constraints": {
        "no_immediate_reassignment": True,
        "cooldown_enforced": True
    },
    "created_at": datetime.utcnow().isoformat()
}

path = ART_CORE10 / "reservation_params.json"
path.write_text(json.dumps(reservation_params, indent=2), encoding="utf-8")

path

PosixPath('../artifact/core10/reservation_params.json')

In [16]:
missing = {
    k: str(v)
    for k, v in BUNDLE_FILES.items()
    if not v.exists()
}

if missing:
    raise FileNotFoundError(
        "Missing required bundle artifacts:\n"
        + "\n".join(f"- {k}: {p}" for k, p in missing.items())
    )

print("✅ all bundle artifacts exist")

✅ all bundle artifacts exist


In [17]:
copied = []

for key, src in BUNDLE_FILES.items():
    dst = BUNDLE_DIR / src.name
    shutil.copy2(src, dst)
    copied.append(dst.name)

copied

['shutdown_contract.json',
 'fallback_params.json',
 'reservation_params.json',
 'core10_06_allocation_policy.json',
 'core10_07_operation_rollout_log.csv',
 'core10_10_metrics_summary.csv']

In [18]:
bundle_metadata = {
    "bundle_id": f"core10_bundle_{datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}",
    "created_at": datetime.utcnow().isoformat(),
    "bundle_path": str(BUNDLE_DIR.resolve()),
    "contents": copied,
    "core_version": "core10",
    "note": "Operational handoff bundle for governed allocation system",
}

In [19]:
META_PATH = BUNDLE_DIR / "bundle_manifest.json"
META_PATH.write_text(
    json.dumps(bundle_metadata, indent=2, ensure_ascii=False),
    encoding="utf-8",
)

print("✅ bundle manifest written:")
print(META_PATH.resolve())

✅ bundle manifest written:
/Users/mac/Desktop/De/Developability_Data/core/artifact/handoff/core10_bundle/bundle_manifest.json


In [20]:
inventory = pd.DataFrame([
    {
        "artifact": k,
        "filename": v.name,
        "size_kb": round(v.stat().st_size / 1024, 2),
    }
    for k, v in BUNDLE_FILES.items()
])

inventory

,artifact,filename,size_kb
0,shutdown_contract,shutdown_contract.json,0.41
1,fallback_params,fallback_params.json,0.34
2,reservation_params,reservation_params.json,0.21
3,allocation_policy,core10_06_allocation_policy.json,3.38
4,operation_rollout_log,core10_07_operation_rollout_log.csv,13.58
5,metrics_summary,core10_10_metrics_summary.csv,0.24


## Handoff Instructions (for receiver)

1. Copy entire `core10_bundle/` directory
2. Treat `bundle_manifest.json` as the authoritative index
3. Load:
   - allocation_policy.json → policy engine
   - shutdown_contract.json → termination guard
   - fallback / reservation params → safety defaults
4. Use:
   - operation_rollout_log.csv
   - metrics_summary.csv
   as **evidence that governance operated**

No notebooks are required to *use* this bundle.
Notebooks are only needed to *re-generate* it.